# Semantic Gravity Mapping - Visualization

This notebook creates interactive visualizations of completed SGM analysis.

**Prerequisites**: You must have already run `semantic_gravity_mapping.ipynb` to generate:
- `semantic_graph_final.gpickle` - The weighted graph
- `topology_metrics.json` - Analysis results

**Visualizations Created**:
1. Interactive network graphs (Pyvis)
2. 2D/3D UMAP manifold projections (Plotly)
3. Convergence heatmaps
4. Asymmetry matrices
5. Island cluster exploration
6. Weight distribution analysis

---

## Cell 1: Setup & Load Results

In [ ]:
import sys
import json
from pathlib import Path

# Detect environment
IN_COLAB = 'google.colab' in sys.modules

print(f"Running in: {'Google Colab' if IN_COLAB else 'Local Environment'}")

if IN_COLAB:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Install dependencies
    print("\nInstalling visualization dependencies...")
    !pip install -q plotly pyvis umap-learn scikit-learn scipy networkx
    
    # Clone repo if needed
    if not Path('/content/align_prompts').exists():
        !git clone https://github.com/YOUR_USERNAME/align_prompts.git /content/align_prompts
    
    sys.path.insert(0, '/content/align_prompts')
    
    print("✓ Colab setup complete")

# Import visualization modules
from align_test.sgm.storage.checkpoint_manager import CheckpointManager
from align_test.sgm.visualization import (
    InteractiveGraphVisualizer,
    ManifoldProjector,
    HeatmapGenerator,
    IslandExplorer
)

print("\n✓ All modules imported successfully!")

## Cell 2: Configuration

In [ ]:
# Configuration
VIZ_CONFIG = {
    # Input files (from Phase 1-3)
    'graph_dir': '/content/drive/MyDrive/sgm_outputs' if IN_COLAB else './data/sgm/graphs',
    'graph_file': 'semantic_graph_final.gpickle',
    'metrics_file': 'topology_metrics.json',
    
    # Output directory
    'output_dir': '/content/drive/MyDrive/sgm_visualizations' if IN_COLAB else './data/sgm/visualizations',
    
    # Visualization settings
    'max_nodes_network': 500,  # For Pyvis performance
    'max_nodes_umap': 1000,    # For UMAP performance
}

print("Configuration:")
for key, value in VIZ_CONFIG.items():
    print(f"  {key}: {value}")

## Cell 3: Load Graph & Analysis Results

In [ ]:
# Initialize checkpoint manager
checkpoint_manager = CheckpointManager(
    checkpoint_dir=VIZ_CONFIG['graph_dir']
)

# Load graph
print("Loading graph...")
graph = checkpoint_manager.load_graph(VIZ_CONFIG['graph_file'].replace('.gpickle', ''))

print(f"✓ Graph loaded: {graph.number_of_nodes():,} nodes, {graph.number_of_edges():,} edges")

# Load analysis results
metrics_path = Path(VIZ_CONFIG['graph_dir']) / VIZ_CONFIG['metrics_file']
print(f"\nLoading analysis results from {metrics_path}...")

with open(metrics_path, 'r') as f:
    results = json.load(f)

print(f"✓ Analysis results loaded")
print(f"  Hubs: {len(results['hubs'])}")
print(f"  Islands: {len(results['islands'])}")
print(f"  Asymmetric pairs: {len(results['asymmetry'])}")

## Cell 4: Interactive Network - Full Graph

In [ ]:
print("=" * 70)
print("CREATING INTERACTIVE NETWORK VISUALIZATION")
print("=" * 70)

# Initialize visualizer
graph_viz = InteractiveGraphVisualizer(
    graph=graph,
    analysis_results=results,
    output_dir=VIZ_CONFIG['output_dir']
)

# Create full network (top 500 nodes)
print(f"\nCreating full network (top {VIZ_CONFIG['max_nodes_network']} nodes by PageRank)...")
network_path = graph_viz.create_full_network(
    filter_top_n=VIZ_CONFIG['max_nodes_network'],
    color_by='pagerank',
    size_by='pagerank'
)

print(f"\n✓ Network visualization created!")
print(f"  Open in browser: {network_path}")

if IN_COLAB:
    from IPython.display import IFrame
    display(IFrame(src=network_path, width=1000, height=600))

## Cell 5: Interactive Network - Hub Network

In [ ]:
print("Creating hub network (top 20 hubs + neighbors)...")
hub_network_path = graph_viz.create_hub_network(
    hub_count=20,
    include_neighbors=True
)

print(f"\n✓ Hub network created: {hub_network_path}")

if IN_COLAB:
    from IPython.display import IFrame
    display(IFrame(src=hub_network_path, width=1000, height=600))

## Cell 6: 2D UMAP Projection

In [ ]:
print("=" * 70)
print("CREATING 2D UMAP PROJECTION")
print("=" * 70)

# Initialize projector
manifold = ManifoldProjector(
    graph=graph,
    analysis_results=results,
    output_dir=VIZ_CONFIG['output_dir']
)

# Compute graph distances (this may take a few minutes)
print("\nComputing graph distances...")
distances, node_list = manifold.compute_graph_distances(
    max_nodes=VIZ_CONFIG['max_nodes_umap']
)

# Project to 2D
print("\nProjecting to 2D with UMAP...")
coords_2d, metadata = manifold.project_2d()

# Create interactive plot
print("\nCreating Plotly 2D visualization...")
fig_2d = manifold.create_plotly_2d(
    coords=coords_2d,
    metadata=metadata,
    color_by='pagerank'
)

# Display
fig_2d.show()

## Cell 7: 3D UMAP Projection

In [ ]:
print("=" * 70)
print("CREATING 3D UMAP PROJECTION")
print("=" * 70)

# Project to 3D
print("\nProjecting to 3D with UMAP...")
coords_3d, metadata_3d = manifold.project_3d()

# Create interactive 3D plot
print("\nCreating Plotly 3D visualization...")
fig_3d = manifold.create_plotly_3d(
    coords=coords_3d,
    metadata=metadata_3d,
    color_by='pagerank'
)

# Display
fig_3d.show()

## Cell 8: Convergence Heatmap

In [ ]:
print("=" * 70)
print("CREATING HEATMAPS")
print("=" * 70)

# Initialize heatmap generator
heatmap_gen = HeatmapGenerator(
    analysis_results=results,
    output_dir=VIZ_CONFIG['output_dir']
)

# Convergence heatmap
print("\nCreating convergence heatmap...")
fig_conv = heatmap_gen.convergence_heatmap()
fig_conv.show()

## Cell 9: Asymmetry Matrix

In [ ]:
print("\nCreating asymmetry matrix...")
fig_asym = heatmap_gen.asymmetry_matrix(top_n=50)
fig_asym.show()

## Cell 10: Hub Centrality Comparison

In [ ]:
print("\nCreating hub centrality comparison...")
fig_hub = heatmap_gen.hub_centrality_comparison()
fig_hub.show()

## Cell 11: Weight Distribution

In [ ]:
print("\nCreating weight distribution plot...")
fig_weight = heatmap_gen.weight_distribution_kde(graph)
fig_weight.show()

## Cell 12: Island Exploration

In [ ]:
print("=" * 70)
print("EXPLORING ISLANDS (ISOLATED CLUSTERS)")
print("=" * 70)

# Initialize island explorer
island_explorer = IslandExplorer(
    graph=graph,
    analysis_results=results,
    output_dir=VIZ_CONFIG['output_dir']
)

# Create island grid
print("\nCreating island grid...")
fig_islands = island_explorer.create_island_grid(max_islands=6)
fig_islands.show()

# Island statistics table
print("\nIsland Statistics:")
df_islands = island_explorer.island_statistics_table()
display(df_islands)

## Cell 13: Summary & Export

In [ ]:
print("=" * 70)
print("VISUALIZATION COMPLETE!")
print("=" * 70)

print("\n📁 All visualizations saved to:")
print(f"   {VIZ_CONFIG['output_dir']}")

print("\n📊 Files created:")
output_dir = Path(VIZ_CONFIG['output_dir'])
if output_dir.exists():
    for file in sorted(output_dir.glob('*.html')):
        print(f"   • {file.name}")

print("\n💡 Next Steps:")
print("   1. Open HTML files in your browser for interactive exploration")
print("   2. Use Pyvis network to zoom, drag, and click nodes")
print("   3. Rotate 3D UMAP plot to see clusters from different angles")
print("   4. Analyze heatmaps to identify patterns and biases")

if IN_COLAB:
    from google.colab import files
    
    download = input("\nDownload all visualizations? (y/n): ")
    if download.lower() == 'y':
        print("\nDownloading...")
        for file in output_dir.glob('*.html'):
            files.download(str(file))
        print("✓ Download complete!")

print("\n" + "=" * 70)
print("Thank you for using Semantic Gravity Mapping Visualization!")
print("=" * 70)